In [ ]:
from sentence_transformers import SentenceTransformer
import torch
# Download from the 🤗 Hub
model = SentenceTransformer("google/embeddinggemma-300m")

# Run inference with queries and documents
query = "Which planet is known as the Red Planet?"
documents = [
    "Venus is often called Earth's twin because of its similar size and proximity.",
    "Mars, known for its reddish appearance, is often referred to as the Red Planet.",
    "Jupiter, the largest planet in our solar system, has a prominent red spot.",
    "Saturn, famous for its rings, is sometimes mistaken for the Red Planet."
]
query_embeddings = model.encode_query(query)
document_embeddings = model.encode_document(documents)
print(query_embeddings.shape, document_embeddings.shape)
# similarities = model.similarity(query_embeddings, document_embeddings)
# print(similarities.shape)

In [ ]:
from hf_olmo import OLMoForCausalLM, OLMoTokenizerFast

if torch.cuda.is_available():
    device = 'cuda'
elif torch.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'


tokenizer = OLMoTokenizerFast.from_pretrained("allenai/OLMo-1B")
sorted_vocab = sorted(tokenizer.get_vocab().items(), key=lambda item: item[1])
dict_sorted_vocab = dict(sorted_vocab)
print(sorted_vocab[:10])

In [ ]:
token_strings = [pair[0] for pair in sorted_vocab]
token_strings[:10]

message for tomorrow: token_strings as the "documents" and each individual token as the query. 

In [ ]:

def create_lookup_table(model): #perform on h100 or higher for improved efficacy.
    similarity_list = [] #will store ordered cos sims
    index_list = [] # will store respective indices
    for token in token_strings:
        similarities = model.similarity(model.encode_query(token), model.encode_document(token_strings))
        similarities[dict_sorted_vocab[token]] = float('-inf') #when passed into toipk, it will be not selected.  we want the tokens with largest cos sim.
        knn = torch.topk(similarities, largest=True, sorted=True) #greatest to least
        similarity_list.append(list(knn[0]))
        index_list.append(list(knn[1]))

    return torch.tensor(similarity_list), torch.tensor(index_list)
